In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
df = pd.read_csv('/content/Financial Distress.csv')
df.dropna(inplace=True)

X = df[['Company', 'Time', 'x1', 'x2', 'x3', 'x4']]
df['Label - Financial Distress'] = pd.cut(df['Financial Distress'], bins=[-float('inf'), 10,28,45,63,80,96,111,float('inf')], labels=[0,1, 2, 3, 4, 5, 6, 7], right=False)
y = df['Label - Financial Distress']

label_encoder = LabelEncoder()
X['Company'] = label_encoder.fit_transform(df['Company'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


learning_rate = 0.001
gamma = 0.95
epsilon = 1.0
episodes = 200

# Create unique integer labels for each action
action_size = len(np.unique(y))
action_labels = np.arange(action_size)


# Train and evaluate the combined DQN model
state_size = X.shape[1]

Decision Tree

In [ ]:
# Train and evaluate the base decision tree model
base_model = DecisionTreeClassifier()
base_model.fit(X_train, y_train)
base_accuracy_dt = round(base_model.score(X_test, y_test), 2)
print("Base Model Accuracy: {:.2f}".format(base_accuracy_dt))
base_recall_dt = round(recall_score(y_test, base_model.predict(X_test), average='macro'),2)
base_precision_dt = round(precision_score(y_test, base_model.predict(X_test), average='macro'),2)
print("Base Model Recall:", base_recall_dt)
print("Base Model Precision:", base_precision_dt)

class DQN:
    def __init__(self, state_size, action_size, learning_rate, gamma, epsilon):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon

        # Build the Q-network (ensemble of decision trees)
        self.model = [DecisionTreeClassifier() for _ in range(self.action_size)]

    def remember(self, state, action, reward, next_state, done):
        pass  # Not needed for decision tree-based approach

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        else:
            q_values = [model.predict_proba(state.reshape(1, -1)) for model in self.model]
            return np.argmax(q_values)

    def replay(self, batch_size):
        pass  # Not needed for decision tree-based approach

    def train(self, X, y, episodes, epochs):
        for episode in range(episodes):
            done = False
            total_reward = 0

            for state in X:
                while not done:
                    action = self.act(state)
                    reward = 1 if (action_labels[action] == y.values).any() else -1

                    total_reward += reward
                    done = True

                    for _ in range(epochs):
                        self.model[action].fit([state], [action_labels[action]])

    def predict(self, X):
        y_pred = np.zeros((len(X), self.action_size))
        for i, state in enumerate(X):
            action = self.act(state)
            y_pred[i, action] = 1
        return y_pred


dqn_agent = DQN(state_size, action_size, learning_rate, gamma, epsilon)
dqn_agent.train(X_train.values, y_train, episodes, 3)
dqn_accuracy_dt = round(np.mean(np.argmax(dqn_agent.predict(X_test.values), axis=1) == y_test), 2)
print("DQN Accuracy: {:.2f}".format(dqn_accuracy_dt))
dqn_recall_dt = round(recall_score(y_test, np.argmax(dqn_agent.predict(X_test.values), axis=1), average='macro'),2)
dqn_precision_dt = round(precision_score(y_test, np.argmax(dqn_agent.predict(X_test.values), axis=1), average='macro'),2)
print("DQN Model Recall:", dqn_recall_dt)
print("DQN Model Precision:", dqn_precision_dt)


# varying epochs
epochs = []
dqn_list = []
for i in range(1,10):
  epochs.append(i)
  dqn_agent.train(X_train.values, y_train, episodes, i)
  dqn_list.append(round(np.mean(np.argmax(dqn_agent.predict(X_test.values), axis=1) == y_test), 2))

plt.plot(epochs,dqn_list, color = '#65cbe9')
plt.xlabel('Number of epochs')
plt.ylabel('Validation Accuracy')
plt.title('Variation of Accuracy with Epochs')
plt.show()

Random Forest

In [ ]:
# Train and evaluate the base random forest model
base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
base_accuracy_rf = round(base_model.score(X_test, y_test),2)
print("Base Model Accuracy: {:.2f}".format(base_accuracy_rf))
base_recall_rf = round(recall_score(y_test, base_model.predict(X_test), average='macro'),2)
base_precision_rf = round(precision_score(y_test, base_model.predict(X_test), average='macro'),2)
print("Base Model Recall:", base_recall_rf)
print("Base Model Precision:", base_precision_rf)



class DQN:
    def __init__(self, state_size, action_size, learning_rate, gamma, epsilon):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon

        # Build the Q-network (ensemble of random forest classifiers)
        self.model = [RandomForestClassifier() for _ in range(self.action_size)]

    def remember(self, state, action, reward, next_state, done):
        pass

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        else:
            q_values = [model.predict_proba(state.reshape(1, -1)) for model in self.model]
            return np.argmax(q_values)

    def replay(self, batch_size):
        pass

    def train(self, X, y, episodes, epochs):
        for episode in range(episodes):
            done = False
            total_reward = 0

            for state in X:
                while not done:
                    action = self.act(state)
                    reward = 1 if (action_labels[action] == y.values).any() else -1

                    total_reward += reward
                    done = True

                    for _ in range(epochs):
                        self.model[action].fit([state], [action_labels[action]])

    def predict(self, X):
        y_pred = np.zeros((len(X), self.action_size))
        for i, state in enumerate(X):
            action = self.act(state)
            y_pred[i, action] = 1
        return y_pred

dqn_agent = DQN(state_size, action_size, learning_rate, gamma, epsilon)
dqn_agent.train(X_train.values, y_train, episodes, 1)
dqn_accuracy_rf = round(np.mean(np.argmax(dqn_agent.predict(X_test.values), axis=1) == y_test),2)
print("DQN Accuracy: {:.2f}".format(dqn_accuracy_rf))
dqn_recall_rf = round(recall_score(y_test, np.argmax(dqn_agent.predict(X_test.values), axis=1), average='macro'),2)
dqn_precision_rf = round(precision_score(y_test, np.argmax(dqn_agent.predict(X_test.values), axis=1), average='macro'),2)
print("DQN Model Recall:", dqn_recall_rf)
print("DQN Model Precision:", dqn_precision_rf)


# varying epochs
epochs = []
dqn_list = []
for i in range(1,10):
  epochs.append(i)
  dqn_agent.train(X_train.values, y_train, episodes, i)
  dqn_list.append(round(np.mean(np.argmax(dqn_agent.predict(X_test.values), axis=1) == y_test), 2))

plt.plot(epochs,dqn_list, color = '#65cbe9')
plt.xlabel('Number of epochs')
plt.ylabel('Validation Accuracy')
plt.title('Variation of Accuracy with Epochs')
plt.show()

Naive Bayes

In [ ]:
# Train and evaluate the base Naive Bayes model
base_model = GaussianNB()
base_model.fit(X_train, y_train)
base_accuracy_nb = round(base_model.score(X_test, y_test),2)
print("Base Model Accuracy: {:.2f}".format(base_accuracy_nb))
base_recall_nb = round(recall_score(y_test, base_model.predict(X_test), average='macro'),2)
base_precision_nb = round(precision_score(y_test, base_model.predict(X_test), average='macro'),2)
print("Base Model Recall:", base_recall_nb)
print("Base Model Precision:", base_precision_nb)


class DQN:
    def __init__(self, state_size, action_size, learning_rate, gamma, epsilon):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon

        # Build the Q-network (ensemble of Naive Bayes models)
        self.model = [GaussianNB() for _ in range(self.action_size)]

    def remember(self, state, action, reward, next_state, done):
        pass

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        else:
            q_values = [model.predict_proba([state])[0] for model in self.model]
            return np.argmax(q_values)

    def replay(self, batch_size):
        pass

    def train(self, X, y, episodes, epochs):
        for episode in range(episodes):
            done = False
            total_reward = 0

            for state in X:
                while not done:
                    action = self.act(state)
                    reward = 1 if (action_labels[action] == y.values).any() else -1

                    total_reward += reward
                    done = True

                    for _ in range(epochs):
                        self.model[action].fit([state], [action_labels[action]])

    def predict(self, X):
        y_pred = np.zeros((len(X), self.action_size))
        for i, state in enumerate(X):
            action = self.act(state)
            y_pred[i, action] = 1
        return y_pred

dqn_agent = DQN(state_size, action_size, learning_rate, gamma, epsilon)
dqn_agent.train(X_train.values, y_train, episodes, 10)
dqn_accuracy_nb = round(np.mean(np.argmax(dqn_agent.predict(X_test.values), axis=1) == y_test),2)
print("DQN Accuracy: {:.2f}".format(dqn_accuracy_nb))
dqn_recall_nb = round(recall_score(y_test, np.argmax(dqn_agent.predict(X_test.values), axis=1), average='macro'),2)
dqn_precision_nb = round(precision_score(y_test, np.argmax(dqn_agent.predict(X_test.values), axis=1), average='macro'),2)
print("DQN Model Recall:", dqn_recall_nb)
print("DQN Model Precision:", dqn_precision_nb)


# varying epochs
epochs = []
dqn_list = []
for i in range(1,10):
  epochs.append(i)
  dqn_agent.train(X_train.values, y_train, episodes, i)
  dqn_list.append(round(np.mean(np.argmax(dqn_agent.predict(X_test.values), axis=1) == y_test), 2))

plt.plot(epochs,dqn_list, color = '#65cbe9')
plt.xlabel('Number of epochs')
plt.ylabel('Validation Accuracy')
plt.title('Variation of Accuracy with Epochs')
plt.show()


Evaluation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

algo = ("Decision Tree", "Random Forest Classifier", "Naive Bayes")
models = {
    'Base Model': (base_accuracy_dt, base_accuracy_rf, base_accuracy_nb),
    'Model with DQN' : (dqn_accuracy_dt, dqn_accuracy_rf, dqn_accuracy_nb),
}

x = np.arange(len(algo))
width = 0.25
multiplier = 0

fig, ax = plt.subplots()

colors = ['#AFD5F0', '#D4FAFA']

for attribute, measurement in models.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute, color=colors[multiplier])
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Validation Accuracy')
ax.set_title('Comparison for various models')
ax.set_xticks(x + width, algo)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 1)

plt.show()
